In [10]:
%pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 46.4 MB 41.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import json
import math
import os
import cv2 #this is from opencv-python
from PIL import Image
    #deal with images: https://pillow.readthedocs.io/en/stable/reference/Image.html
import numpy as np
from keras import layers
from keras.applications import DenseNet201
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score
import scipy
from tqdm import tqdm
import tensorflow as tf
from keras import backend as K
import gc
from functools import partial
from sklearn import metrics
from collections import Counter
import json
import itertools

In [6]:
DIR='data/train/benign'
os.listdir(DIR)

['SOB_B_A-14-29960CD-400-009.png',
 '.DS_Store',
 'SOB_B_A-14-29960CD-400-008.png',
 'SOB_B_A-14-22549G-400-005.png',
 'SOB_B_A-14-22549G-400-011.png',
 'SOB_B_A-14-22549G-400-010.png',
 'SOB_B_A-14-22549G-400-004.png',
 'SOB_B_A-14-22549G-400-012.png',
 'SOB_B_A-14-22549G-400-006.png',
 'SOB_B_A-14-22549G-400-007.png',
 'SOB_B_A-14-22549G-400-013.png',
 'SOB_B_A-14-22549G-400-017.png',
 'SOB_B_A-14-22549G-400-003.png',
 'SOB_B_A-14-22549G-400-002.png',
 'SOB_B_A-14-22549G-400-016.png',
 'SOB_B_A-14-22549CD-400-029.png',
 'SOB_B_A-14-22549G-400-028.png',
 'SOB_B_A-14-22549G-400-014.png',
 'SOB_B_A-14-22549G-400-015.png',
 'SOB_B_A-14-22549G-400-001.png',
 'SOB_B_A-14-22549G-400-029.png',
 'SOB_B_A-14-22549CD-400-028.png',
 'SOB_B_A-14-22549CD-400-025.png',
 'SOB_B_A-14-22549G-400-024.png',
 'SOB_B_A-14-22549G-400-018.png',
 'SOB_B_A-14-22549G-400-019.png',
 'SOB_B_A-14-22549G-400-025.png',
 'SOB_B_A-14-22549CD-400-030.png',
 'SOB_B_A-14-22549CD-400-026.png',
 'SOB_B_A-14-22549G-400-027

In [9]:
PATH = os.path.join(DIR,'SOB_B_A-14-29960CD-400-003.png')
_, ftype = os.path.splitext(PATH)
print(_)
print(ftype)

data/train/benign/SOB_B_A-14-29960CD-400-003
.png


In [12]:
def Dataset_loader(DIR, RESIZE, sigmaX=10):
    IMG = []
    read = lambda imname: np.asarray(Image.open(imname).convert("RGB")) #create a function to read images, using 'Image' from PIL
    for IMAGE_NAME in tqdm(os.listdir(DIR)): #the qddm is just to show the process bar
        PATH = os.path.join(DIR,IMAGE_NAME)
        _, ftype = os.path.splitext(PATH) #this is filter only files with .png because there are system's files in the folder too
        if ftype == ".png":
            img = read(PATH)        
            img = cv2.resize(img, (RESIZE,RESIZE)) #this is 224x224, the original size is 400x400          
            IMG.append(np.array(img)) #append to list
    return IMG

benign_train = np.array(Dataset_loader('data/train/benign',224))
malign_train = np.array(Dataset_loader('data/train/malignant',224))
benign_test = np.array(Dataset_loader('data/validation/benign',224))
malign_test = np.array(Dataset_loader('data/validation/malignant',224))

100%|██████████| 32/32 [00:00<00:00, 46.04it/s]
